In [162]:
import os;
import json;
import builtins
import pandas as pd;

dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge'
testcase = 'fb_merge-ratio10_1'
test_dir = os.path.join(dir, testcase)
results_dir = os.path.join(test_dir, 'outputs', 'results')
csv_dir = os.path.join(test_dir, 'csv')
os.makedirs(csv_dir, exist_ok=True)

runs = [os.path.join(results_dir, run) for run in os.listdir(results_dir)]
print(runs)

['/home/chesetti/Repos/learned_merge_cleanup/sponge/fb_join-ratio10_1/outputs/results/run0', '/home/chesetti/Repos/learned_merge_cleanup/sponge/fb_join-ratio10_1/outputs/results/run1', '/home/chesetti/Repos/learned_merge_cleanup/sponge/fb_join-ratio10_1/outputs/results/run2']


In [163]:
# Walk all the json files and put them in a dataframe

test_results = []
for run in runs:
    for test_result_file in os.listdir(run):
        json_file = builtins.open(os.path.join(run, test_result_file))
        test_result = json.load(json_file)
        test_result['run'] = run
        test_results.append(test_result)
        json_file.close()
test_dataframe = pd.json_normalize(test_results)
print(test_dataframe.columns)

        

Index(['command', 'run', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_index_build_duration_ns', 'result.inner_index_size',
       'result.outer_disk_fetch', 'spec.algo', 'spec.algo_name',
       'spec.common_key', 'spec.index.search', 'spec.index.type',
       'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk', 'spec.index.leaf_size_in_pages'],
      dtype='object')


# Reports

### Overall Duration

In [164]:
overall_duration = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.duration_ns', aggfunc='median')
overall_duration.to_csv(os.path.join(csv_dir, 'duration_sec.csv'))
print(overall_duration)

spec.algo_name   btree1024  btree2048  btree256  btree512   hashJoin  \
spec.common_key                                                        
10                48183758   49442459  53673518  49180968  219008565   
20                27977632   28379421  32053963  28994890  201621948   
30                21052902   21266836  24600014  21982897  211452987   
40                17667601   17722639  20912587  18874960  184065463   
50                15378972   15418314  18586705  16333217  208041184   
60                14161691   13971369  17098943  15013708  186681895   
70                13081469   12810237  15937983  13884355  174617742   
80                12242349   12180351  15165192  13088427  157458183   
90                11655927   11376269  14185017  12704900  149363409   
100               11133244   11123294  13572844  12101513  175720112   

spec.algo_name    pgm1024    pgm128    pgm256    pgm512     pgm64        sj  \
spec.common_key                                         

In [165]:
inner_index_size = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_index_size', aggfunc='median')
data = {}
data['indexes'] = []
data['memory_bytes'] = []
for index in inner_index_size:
    data['indexes'].append(index)
    data['memory_bytes'].append(inner_index_size[index].mean())
inner_index_size = pd.DataFrame(data=data)
inner_index_size.to_csv(os.path.join(csv_dir, 'inner_index_size.csv'))
print(inner_index_size)

      indexes  memory_bytes
0   btree1024       37072.0
1   btree2048       12352.0
2    btree256      131832.0
3    btree512       70032.0
4    hashJoin      161584.0
5     pgm1024         152.0
6      pgm128        3696.0
7      pgm256        1000.0
8      pgm512         312.0
9       pgm64       13488.0
10         sj      161584.0
11        sj2      161584.0


In [166]:
inner_index_build_duration = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_index_build_duration_ns', aggfunc='median')
data = {}
data['indexes'] = []
data['build_duration'] = []
for index in inner_index_build_duration:
    data['indexes'].append(index)
    data['build_duration'].append(inner_index_build_duration[index].mean())
inner_index_build_duration = pd.DataFrame(data=data)
inner_index_build_duration.to_csv(os.path.join(csv_dir, 'inner_index_build_duration_ns.csv'))

In [167]:
inner_index_fetch = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_disk_fetch', aggfunc='median')
print(inner_index_fetch)

spec.algo_name   btree1024  btree2048  btree256  btree512  hashJoin  pgm1024  \
spec.common_key                                                                
10                    1944        486      7773      3887      7774      864   
20                    1944        486      7773      3887      7774      864   
30                    1944        486      7771      3887      7774      864   
40                    1944        486      7759      3887      7774      864   
50                    1944        486      7723      3886      7774      864   
60                    1944        486      7663      3886      7774      864   
70                    1944        486      7576      3887      7774      864   
80                    1944        486      7440      3880      7774      864   
90                    1944        486      7326      3879      7774      864   
100                   1944        486      7160      3860      7774      864   

spec.algo_name   pgm128  pgm256  pgm512

In [168]:
result_checksum = (test_dataframe[['spec.common_key', 'spec.algo_name', 'result.checksum']].sort_values(by=['spec.common_key', 'spec.algo_name'])) #.loc[test_dataframe['spec.common_key']=='10'])
for common_key in sorted(test_dataframe['spec.common_key'].unique()):
    checksums = result_checksum.loc[result_checksum['spec.common_key'] == common_key]
    unique_checksums = checksums['result.checksum'].unique()
    if (len(unique_checksums) == 1):
        print(f"common_key: {common_key} checksum: {unique_checksums} OK")
    else:
        print(f"common_key: {common_key} checksums don't match")
print(result_checksum.loc[result_checksum['spec.common_key'] == 40])
print(result_checksum.loc[result_checksum['spec.common_key'] == 40]['result.checksum'].unique())

common_key: 10 checksum: ['3AEE37A7397FFD1C7241C4244A239156'] OK
common_key: 20 checksum: ['D815BB1CE4DF453E2E3613FBC5353EE8'] OK
common_key: 30 checksum: ['335E583B7853DC9BAEE09CE0DA61118E'] OK
common_key: 40 checksum: ['551DBFF45F1B2DEE9ACA32C2A84C5CBD'] OK
common_key: 50 checksum: ['0FB542B7155B8D61A2FE47A4C2E7BC6C'] OK
common_key: 60 checksum: ['877B3D3B677FBF841110AA99D76C5D38'] OK
common_key: 70 checksum: ['61B2F0607274D31841C30A2679F687BA'] OK
common_key: 80 checksum: ['C618430BDC57794D14E321BDEE4BD4A9'] OK
common_key: 90 checksum: ['2022A49803D73CB6B6674CAB9F46C57B'] OK
common_key: 100 checksum: ['126B013F8833241E9558B8D0B1B7BBBA'] OK
     spec.common_key spec.algo_name                   result.checksum
83                40      btree1024  551DBFF45F1B2DEE9ACA32C2A84C5CBD
203               40      btree1024  551DBFF45F1B2DEE9ACA32C2A84C5CBD
323               40      btree1024  551DBFF45F1B2DEE9ACA32C2A84C5CBD
66                40      btree2048  551DBFF45F1B2DEE9ACA32C2A84C5CBD